### Szövegfeldolgozás

In [91]:
ugyfel_list = list()
diszpecser_list = list()

count = 0

line_num = 9
greet_strip = 2

for file_name in range(1,100):
    first_n = 0

    count += 1
    if (count % 20 == 0):
        print("{0} file kész van".format(count))
    ugyfel_flag = 0
    diszpecser_flag = 0            
    t_ugyf_l = list()
    t_disz_l = list()
    with open("C:\Aktuális\Egyetem\Önlab\TextGrid\{0}.TextGrid".format("%04d" % file_name), encoding="utf-16-be", errors="ignore") as f:
        for line in f:
            temp = line.replace('\x00', "").strip()
            if (len(temp.split()) != 0):

                if ("name" in temp and "diszpecser" in temp):
                    ugyfel_flag = 0
                    diszpecser_flag = 1
                elif("name" in temp and "ugyfel" in temp):
                    diszpecser_flag = 0
                    ugyfel_flag = 1
                if(temp.split()[0] == "text" and temp.split()[2].strip("\"") is not ""):
                    if(diszpecser_flag == 1):
                        t_disz_l.append(temp[8:-1])
                        # print("a diszpécser azt mondta: " + temp[8:-1])
                    elif(ugyfel_flag == 1):
                        first_n += 1
                        if first_n > line_num:
                            break
                        if first_n > greet_strip:
                            t_ugyf_l.append(temp[8:-1])
                        # print("az ugyfél azt mondta: " + temp[8:-1])
    if len(t_ugyf_l) == line_num-greet_strip:
        ugyfel_list.append(t_ugyf_l)
    diszpecser_list.append(t_disz_l)
    
# print(len(diszpecser_list))
# print(diszpecser_list[3])

20 file kész van
40 file kész van
60 file kész van
80 file kész van
100 file kész van
120 file kész van
140 file kész van
160 file kész van
180 file kész van
200 file kész van
220 file kész van
240 file kész van
260 file kész van
280 file kész van
300 file kész van
320 file kész van
340 file kész van
360 file kész van
380 file kész van
400 file kész van
420 file kész van
440 file kész van
460 file kész van
480 file kész van
500 file kész van
520 file kész van
540 file kész van
560 file kész van
580 file kész van
600 file kész van
620 file kész van
640 file kész van
660 file kész van
680 file kész van
700 file kész van
720 file kész van
740 file kész van
760 file kész van
780 file kész van
800 file kész van
820 file kész van
840 file kész van
860 file kész van
880 file kész van
900 file kész van
920 file kész van
940 file kész van
960 file kész van
980 file kész van
1000 file kész van
1020 file kész van


In [92]:
print(len(ugyfel_list))

945


In [93]:
stop = list()

with open("stopwords-hu.txt", encoding="utf-16-be", errors="ignore") as f:
    for line in f:
        stop.append(line.strip())
stop.append("a")

### LDA modell alkalmazása

https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

a listák str-é alakítása (minden fájl egy str, az egymáshoz tartozó diszpécser és ügyfél szövegeket is konkatenálva)

In [94]:
doc_complete = list()

for i in range(len(ugyfel_list)):
    temp = ""
    for line in ugyfel_list[i]:
        temp += line +" "
    # for line in diszpecser_list[i]:
    #     temp += line +" "
    doc_complete.append(temp)
    

Cleaning is an important step before any text mining task, in this step, we will remove the punctuations, stopwords and normalize the corpus.

In [99]:
from nltk.corpus import stopwords 
from nltk.stem.snowball import SnowballStemmer
import string
import subprocess

# stop = set(stopwords.words('hungarian'))
exclude = set(string.punctuation) 
stemmer = SnowballStemmer("hungarian")

def lanc_stem(i, string):
    clean_str = list()
    with open("in.txt", "w",encoding = "utf-8", errors="ignore") as f:
        f.write(string)
    subprocess.call(['java', '-Xmx1G', '-jar', 'magyarlanc-3.0.jar', '-mode', 'morphparse', '-input', 'in.txt', '-output', 'out.txt'])
    with open("out.txt", encoding = "utf-8", errors="ignore") as f:
        for line in f:
            if(len(line.split()) != 0):
                clean_str.append(line.split()[1])
    print("{0}. fájl kész van".format(i))
    return clean_str
            
def clean(doc, i=0):
    punc_free = ''.join(ch for ch in doc if ch not in exclude)
    stop_free = " ".join([i for i in punc_free.lower().split() if i not in stop])
    # Magyarlánc
    normalized = lanc_stem(i, stop_free)
    # Snowball Stemmer
    # normalized = " ".join(stemmer.stem(word) for word in stop_free.split())
    
    return normalized
# Magyarlánc
doc_clean = [clean(doc, i) for i, doc in enumerate(doc_complete)]  
# Snowball Stemmer
# doc_clean = [clean(doc).split() for doc in doc_complete]

0. fájl kész van
1. fájl kész van
2. fájl kész van
3. fájl kész van
4. fájl kész van
5. fájl kész van
6. fájl kész van
7. fájl kész van
8. fájl kész van
9. fájl kész van
10. fájl kész van
11. fájl kész van
12. fájl kész van
13. fájl kész van


KeyboardInterrupt: 

All the text documents combined is known as the corpus. To run any mathematical model on text corpus, it is a good practice to convert it into a matrix representation. LDA model looks for repeating term patterns in the entire DT matrix. Python provides many great libraries for text mining practices, “gensim” is one such clean and beautiful library to handle text data. It is scalable, robust and efficient. Following code shows how to convert a corpus into a document-term matrix.

In [96]:
# Importing Gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Next step is to create an object for LDA model and train it on Document-Term matrix. The training also requires few parameters as input which are explained in the above section. The gensim module allows both LDA model estimation from a training corpus and inference of topic distribution on new, unseen documents.

In [97]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=50)

In [98]:
print(ldamodel.print_topics(num_topics=4, num_words=5))

[(0, '0.019*"tud" + 0.017*"szeretne" + 0.013*"szerződés" + 0.010*"száml" + 0.010*"kapt"'), (1, '0.026*"szeretne" + 0.018*"szerződés" + 0.017*"ö" + 0.014*"tud" + 0.014*"kapt"'), (2, '0.023*"szeretne" + 0.017*"szerződés" + 0.012*"invitel" + 0.011*"ö" + 0.008*"intern"'), (3, '0.027*"szeretne" + 0.017*"tud" + 0.012*"szolgáltatás" + 0.011*"telefon" + 0.011*"száml"')]


In [ ]:
doc1_clean = clean(doc_complete[1]).split()
doc1_bow =dictionary.doc2bow(doc1_clean)

ldamodel.get_document_topics(doc1_bow)